In [1]:
%pip install "git+https://github.com/ScierKnave/TorchMPS.git"
!git clone "https://github.com/ScierKnave/honor_project.git"
import requests
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
%pip install torchmetrics
from torchmetrics.classification import MulticlassAccuracy
from torchmps import MPS
import math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ScierKnave/TorchMPS.git to /tmp/pip-req-build-1a5fb3jx
  Running command git clone --filter=blob:none --quiet https://github.com/ScierKnave/TorchMPS.git /tmp/pip-req-build-1a5fb3jx
  Resolved https://github.com/ScierKnave/TorchMPS.git to commit f716a08e15d0af50dbfdfc435ab9604e82562ea3
  Preparing metadata (setup.py) ... done
  Created wheel for torchmps: filename=torchmps-0.1.0-py3-none-any.whl size=59681 sha256=0fbe2bab859c92f83bdde7d4140722d4b8c17dea982d9819a95fa775b91819e5
  Stored in directory: /tmp/pip-ephem-wheel-cache-6cip38a3/wheels/7c/97/ed/e7cfb0c73fec613f09c365db5a9b684875fdb85e8944240efc
Successfully built torchmps
Cloning into 'honor_project'...
remote: Enumerating objects: 883, done.
remote: Counting objects: 100% (172/172), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 883 (delta 111), reused 58 (delta 30), pack-reused 7

# Hyperparameters

In [2]:
# Hardware hyperparameters
chosen_device = torch.device('cuda' 
if torch.cuda.is_available() else 'cpu')

# Data hyperparameters
nb_train_HP = 2000
nb_test_HP = 500
batch_sz_HP = 150
batch_sz_HP = min(batch_sz_HP, nb_train_HP)
nb_classes_HP = 10

# Student hyperparameters
# MPS parameters
bond_dim_HP = 10
# USING CUSTOM FEATURE MAP WITH FORK?: YES

# Training parameters
nepochs_student_HP = 30 
student_lr_HP = 1e-4
student_reg_HP = 0
student_loss_HP = nn.CrossEntropyLoss()


# Premilinaries: Importing the data and utils subroutines

In [3]:

# Import the *common* mnist train set and create a batch iterator for it.
train_set = torch.load('/content/honor_project/src/datasets/train_mnist.pt')
train_iterator = torch.utils.data.DataLoader(
    dataset = train_set, 
    sampler = torch.utils.data.SubsetRandomSampler(range(nb_train_HP)),
    batch_size=batch_sz_HP
    )

# Import the mnist *common* test set and create a batch iterator for it.
test_set = torch.load('/content/honor_project/src/datasets/test_mnist.pt')
test_iterator = torch.utils.data.DataLoader(
    dataset = test_set, 
    sampler = torch.utils.data.SubsetRandomSampler(range(nb_test_HP)),
    batch_size = batch_sz_HP
    )

In [4]:
# Returns the validation set classification accuracy
# of the given input model (this is a higher order function)
def get_acc(model, iterator, dataset_size):
    # Get the validation set classification accuracy
    total_good_classifications = 0
    acc_metric = MulticlassAccuracy(num_classes=nb_classes_HP).to(chosen_device)
    for (x_mb, y_mb) in iterator:
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)
        # Add the number of datapoints we classified right to the total
        batch_size = x_mb.size()[0]
        y_hat = model(x_mb)
        batch_good_classifications = batch_size * acc_metric(y_hat, y_mb)
        total_good_classifications += batch_good_classifications
    return total_good_classifications / dataset_size # divide by total size

# Training the student model

In [5]:
def train_a_student():
  '''
  Trains a student model.
  '''

  # Initialize the MPS module
  student = MPS(
      input_dim = 28 ** 2,
      output_dim = 10,
      bond_dim = bond_dim_HP
  ).to(chosen_device) 
  #student.register_feature_map(feature_map_HP)

  # Instantiate the optimizer and softmax
  student_optimizer = torch.optim.Adam(
      student.parameters(), lr = student_lr_HP, weight_decay = student_reg_HP
  )

  # Used on the inputs before the loss function
  LogSoftmax = nn.LogSoftmax(dim=1)

  # Create an array to store the val loss
  # of the student at each epoch
  stud_test_loss = np.array([])
  stud_train_loss = np.array([])

  softmax = nn.Softmax(dim=1)

  stud_train_loss = np.array([])
  stud_test_loss = np.array([])

  for epoch in range(nepochs_student_HP):
      '''
      Training loop through the epochs.
      '''
      for (x_mb, y_mb) in train_iterator:
          # Flatten the MNIST images, which come in matrix form
          x_mb = x_mb.reshape(-1, 784).to(chosen_device)
          y_mb = y_mb.to(chosen_device)

          student_logits = student(x_mb) 

          # Backpropagation
          loss = student_loss_HP( student_logits, y_mb)

          student_optimizer.zero_grad()
          loss.backward()
          student_optimizer.step()

      # Get accuracy over all test and training data for current epoch
      train_current_accuracy = round( get_acc(student, train_iterator, nb_train_HP).item(), 4)
      test_current_accuracy = round( get_acc(student, test_iterator, nb_test_HP).item(), 4)
      stud_train_loss = np.append(stud_train_loss, train_current_accuracy)
      stud_test_loss = np.append(stud_test_loss, test_current_accuracy)
  return(stud_train_loss, stud_test_loss)


# Repeat the training process in order to get the variance
global_stud_test_loss = np.array([])
global_stud_train_loss = np.array([])
for i in range(20):
  print("\n Training and testing of model ", i+1, " in progress...")
  (stud_train_loss, stud_test_loss) = train_a_student()
  print("Train loss: ", stud_train_loss)
  print("Test loss: ", stud_test_loss)
  if (i == 0):
    global_stud_train_loss = stud_train_loss
    global_stud_test_loss = stud_test_loss
  else:
    global_stud_train_loss = np.vstack((global_stud_train_loss, stud_train_loss))
    global_stud_test_loss = np.vstack((global_stud_test_loss, stud_test_loss))




 Training and testing of model  1  in progress...
Train loss:  [0.1    0.1209 0.3851 0.5417 0.6169 0.7421 0.8319 0.8413 0.8728 0.8789
 0.904  0.9265 0.928  0.9397 0.9546 0.9413 0.9538 0.9456 0.9633 0.9725
 0.9656 0.9605 0.9709 0.9786 0.978  0.9763 0.9868 0.9908 0.993  0.9961]
Test loss:  [0.1    0.126  0.3874 0.51   0.6091 0.7569 0.8409 0.8523 0.8507 0.8339
 0.8607 0.9004 0.8839 0.8719 0.9026 0.8725 0.9003 0.8761 0.8993 0.9013
 0.9099 0.8882 0.8941 0.914  0.9198 0.9103 0.9141 0.9275 0.9085 0.9088]

 Training and testing of model  2  in progress...
Train loss:  [0.1    0.1917 0.2096 0.1574 0.2688 0.4394 0.5415 0.675  0.7603 0.8243
 0.8682 0.8956 0.8661 0.9122 0.9285 0.9299 0.9354 0.9354 0.9549 0.9585
 0.9636 0.9618 0.9682 0.9769 0.9734 0.9803 0.9849 0.9736 0.9831 0.9932]
Test loss:  [0.1    0.2007 0.2041 0.171  0.2592 0.4283 0.5607 0.6876 0.7689 0.8186
 0.8818 0.8852 0.8579 0.8747 0.888  0.8988 0.8565 0.8993 0.8837 0.9041
 0.9076 0.9112 0.9179 0.9063 0.9026 0.9214 0.9164 0.9122 0.9164 

In [6]:
# Print the final results and save them for the report.
print("Final results")
print("Train loss: ", global_stud_train_loss)
print("Test loss: ", global_stud_test_loss)

np.save('20x_mps_trainloss_10bd', stud_train_loss)
np.save('20x_mps_testloss_10bd', stud_test_loss)


Final results
Train loss:  [[0.1    0.1209 0.3851 0.5417 0.6169 0.7421 0.8319 0.8413 0.8728 0.8789
  0.904  0.9265 0.928  0.9397 0.9546 0.9413 0.9538 0.9456 0.9633 0.9725
  0.9656 0.9605 0.9709 0.9786 0.978  0.9763 0.9868 0.9908 0.993  0.9961]
 [0.1    0.1917 0.2096 0.1574 0.2688 0.4394 0.5415 0.675  0.7603 0.8243
  0.8682 0.8956 0.8661 0.9122 0.9285 0.9299 0.9354 0.9354 0.9549 0.9585
  0.9636 0.9618 0.9682 0.9769 0.9734 0.9803 0.9849 0.9736 0.9831 0.9932]
 [0.1    0.3886 0.6493 0.6818 0.8061 0.8361 0.86   0.8945 0.9101 0.9066
  0.9212 0.9379 0.9356 0.9362 0.9457 0.9625 0.954  0.954  0.9547 0.9669
  0.9718 0.982  0.9844 0.9869 0.9832 0.9297 0.9836 0.9785 0.9822 0.9915]
 [0.171  0.1809 0.3102 0.5221 0.693  0.7922 0.8511 0.859  0.8564 0.8957
  0.8936 0.9006 0.9044 0.9389 0.9402 0.9572 0.9575 0.9627 0.9588 0.9614
  0.9665 0.9639 0.9536 0.9852 0.9867 0.9762 0.9397 0.9843 0.9706 0.9854]
 [0.1612 0.1    0.1    0.2366 0.4018 0.6606 0.7544 0.8148 0.873  0.8921
  0.9083 0.9232 0.9325 0.9389 0.9